In [ ]:
import requests
import pandas as pd
import hvplot.pandas
import json
import io
import holoviews as hv
import geoviews as gv
import numpy as np
import geopandas as gp
import searvey
from dateutil.relativedelta import *

hv.extension('bokeh')

In [2]:
# Define the base URL for the API
sl_api  = "https://webcritech.jrc.ec.europa.eu/SeaLevelsDb/api/Group/"
tad_api = "https://webcritech.jrc.ec.europa.eu/TAD_server/api/Groups/Get"

api_device = "https://webcritech.jrc.ec.europa.eu/api/Device/"

# Fetch the list of providers
sl_response = requests.get(sl_api)
tad_response = requests.get(tad_api)
sl_data = json.loads(sl_response.text)  # Parse the JSON response
tad_data = json.loads(tad_response.text)  # Parse the JSON response

## Data availability

### SeaLevelDB stations

In [ ]:
# Initialize a list to hold all station data
SL_data = []

# Loop through the list of providers and fetch their stations
for provider in sl_data:
    provider_name = provider['Name']
    stations_url = sl_api + provider_name + "/Devices"
    stations_response = requests.get(stations_url)
    stations_data = json.loads(stations_response.text)  # Parse the JSON response
    # Loop through each station and extract data
    for device in stations_data:
        station_data = {
            'Provider': provider_name,
            'Id': device['Id'],
            'Name': device['Name'],
            'lat': device['Lat'],
            'lon': device['Lon'],
            'LastAccessStatus': device['CurrentStatus']['LastAccessStatus'],
            'LastDate': device['CurrentStatus']['LastDate'],
            'LastDate': device['CurrentStatus']['LastDate'],
            'State': device['CurrentStatus']['State'],
            'SyncStatus': device['CurrentStatus']['SyncStatus'],
            'FileType': device['FileType'],
            'GroupId': device['GroupId'],
            'MovAvgNp': device['MovAvgNp'],
            'Notes': device.get('Notes'),  # Use .get() to handle missing keys
            'Source': device['Source']
            # Add other fields as necessary
        }
        SL_data.append(station_data)
SL_df = pd.DataFrame(SL_data)
SL_df
SL_df.to_csv('SeaLevelDb_stations.csv')
SL_df.groupby(['Provider','State']).count()[['Id']].hvplot.bar(
    rot = 90,
    logy = True,
    ylim = [0.5, 2000], 
    grid = True
).opts(
    width = 1200,
    height = 800,
    title = "SeaLevelDb data availability"
)

TAD_data Stations

In [ ]:
TAD_data = []
# Loop through the list of providers and fetch their stations
for provider in tad_data:
    provider_name = provider['Name']
    stations_url = f"https://webcritech.jrc.ec.europa.eu/TAD_server/api/Groups/GetGeoJSON?group={provider_name}&maxLatency=10000000"
    stations_response = requests.get(stations_url)
    stations_data = json.loads(stations_response.text)  # Parse the JSON response
    if isinstance(stations_data, dict): stations_data = [stations_data]
    # Loop through each station and extract data
    if len(stations_data)>0:
        for device in stations_data[0]['features']:
            properties = device.get('properties', {})
            geometry = device.get('geometry', {})
            coordinates = geometry.get('coordinates', [None, None])
            latency = properties.get('Latency', {}).get('Literal')
            last_data_date_str = properties.get('LastData', {}).get('Date')
            last_data_date = pd.Timestamp(last_data_date_str)
            
            # Determine if the measurement is within the last week
            state = "active" if last_data_date and (pd.Timestamp.now() - last_data_date) <= pd.Timedelta(days=7) else "inactive"

            station_data = {
                'Provider': properties.get('Provider'),
                'Id': device.get('id'),
                'Name': properties.get('Name'),
                'lat': coordinates[1],
                'lon': coordinates[0],
                'LastAccessStatus': properties.get('LastData', {}).get('Date'),
                'LastDate': last_data_date,
                'SyncStatus': latency,
                'State': state,
                'FileType': None,  # Update this if there is a corresponding field in the new data
                'GroupId': None,  # Update this if there is a corresponding field in the new data
                'MovAvgNp': None,  # Update this if there is a corresponding field in the new data
                'Notes': properties.get('Notes'),  # Use .get() to handle missing keys
                'Source': None
                # Add other fields as necessary
            }
            TAD_data.append(station_data)
TAD_df = pd.DataFrame(TAD_data)
TAD_df
TAD_df.to_csv('TAD_stations.csv')
TAD_df.groupby(['Provider','State']).count()[['Id']].hvplot.bar(
    rot = 90,
    logy = True,
    ylim = [0.5, 2000], 
    grid = True
).opts(
    width = 1600,
    height = 800,
    title = "SeaLevelDb data availability"
)

Reference stations already in use in Seareport: 

In [5]:
def get_stofs2d_meta():
    stofs2d = pd.read_csv(
        "https://polar.ncep.noaa.gov/stofs/data/stofs_2d_glo_elev_stat_v2_1_0",
        names=["coords", "name"],
        sep="!",
        header=None,
        skiprows=1
    )
    stofs2d = stofs2d.assign(
        lon=stofs2d.coords.str.split("\t", n=1).str[0].astype(float),
        lat=stofs2d.coords.str.strip().str.rsplit("\t", n=1).str[1].astype(float),
        stofs2d_name=stofs2d.name.str.strip(),
    ).drop(columns=["coords", "name"])
    return stofs2d


def get_ioc_meta() -> gp.GeoDataFrame:
    meta_web = searvey.get_ioc_stations().drop(columns=["lon", "lat"])
    meta_api = (
        pd.read_json(
            "http://www.ioc-sealevelmonitoring.org/service.php?query=stationlist&showall=all"
        )
        .drop_duplicates()
        .drop(columns=["lon", "lat"])
        .rename(columns={"Code": "ioc_code", "Lon": "lon", "Lat": "lat"})
    )
    merged = pd.merge(
        meta_web,
        meta_api[["ioc_code", "lon", "lat"]].drop_duplicates(),
        on=["ioc_code"],
    )
    return merged

def get_coops_meta() -> gp.GeoDataFrame: 
    coops = searvey.get_coops_stations()
    coops['lon'] = coops['geometry'].x
    coops['lat'] = coops['geometry'].y
    coops = coops.drop(columns='geometry')
    return coops

def merge_ioc_and_stofs(ioc: pd.DataFrame, stofs2d: pd.DataFrame, coops = pd.DataFrame) -> pd.DataFrame:
    stations = pd.concat((ioc, stofs2d), ignore_index=True)
    stations = stations.assign(unique_id=stations.ioc_code.combine_first(stations.stofs2d_name))
    stations = pd.concat((stations, coops),ignore_index=True)
    stations = stations.assign(unique_id=stations.unique_id.combine_first(stations.nws_id))
    return stations

ioc = get_ioc_meta()
stofs2d = get_stofs2d_meta()
coops = get_coops_meta()
stations = merge_ioc_and_stofs(ioc=ioc, stofs2d=stofs2d, coops = coops.drop(columns="removed"))
stations['is_sat'] = stations.unique_id.str.contains('SA')
stations['source'] = "stofs"
stations.loc[~stations.ioc_code.isna(), 'source'] = "ioc"
stations.loc[~stations.nws_id.isna(), 'source'] = "coops"

Compare all stations

In [ ]:
plot1 = stations.drop(columns='geometry')[~stations['is_sat']].hvplot.points(
    x = 'lon',
    y='lat',
    c = 'source',
    cmap = 'fire',
    geo = True,
    s = 20,
    legend = False
)
plot2 = pd.concat([SL_df,TAD_df]).hvplot.points(
    x = 'lon',
    y='lat',
    c = 'Provider',
    line_color = 'k',
    line_width = 0.25,
    cmap = 'glasbey',
    geo = True,
    s=150,
    tiles="OSM",
)

(plot2 * plot1).opts(
    width = 1400,
    height = 1070
)

Let's compare now with the data availability

In [ ]:
SL_df['LastDate'] =  SL_df.apply(lambda x:pd.Timestamp(x['LastDate']).tz_localize(None), axis = 1)
TAD_df['LastDate'] = TAD_df.apply(lambda x:pd.Timestamp(x['LastDate']).tz_localize(None), axis = 1)
SL_df.loc[SL_df['LastDate']>pd.Timestamp.now(), 'LastDate']= pd.Timestamp.now()
SL_df.loc[SL_df['LastDate']<pd.Timestamp(2010,1,1), 'LastDate']= pd.Timestamp(2010,1,1)
SL_df.LastDate.hvplot.hist(bins=100).opts(title = "SeaLevelDb")
TAD_df.LastDate.hvplot.hist().opts(title = "TAD Server")

In [ ]:
SL_df['period_inactive'] = SL_df.apply(lambda x:(pd.Timestamp.now() - pd.Timestamp(x['LastDate']).tz_localize(None)).total_seconds()/3600/24/365.25, axis = 1)
TAD_df['period_inactive'] = TAD_df.apply(lambda x:(pd.Timestamp.now() - pd.Timestamp(x['LastDate']).tz_localize(None)).total_seconds()/3600/24/365.25, axis = 1)

all_stations = pd.concat([SL_df,TAD_df])
all_stations.period_inactive.hvplot.hist().opts(title = "years inactive")

In [9]:
all_stations.to_csv('TAD_SLDB_stations.csv')

In [ ]:
plot_3 = SL_df.hvplot.points(
    x = 'lon',
    y='lat',
    c = 'period_inactive',
    s = 50,
    geo = True,
    cmap = 'fire_r',
    tiles="OSM",
).opts(height = 550, title = 'years inactive')

((plot2 * plot1).opts(height=550, title = 'Provider') + plot_3)

Finnish (FMI) and Indonesian stations (BIG) seem to be exclusive on WebCritech Server. 

Although they stopped recording data since January 2020. 

In [ ]:
plot_4 = TAD_df.hvplot.points(
    x = 'lon',
    y='lat',
    c = 'period_inactive',
    line_color = "k",
    line_width = 0.25,
    cmap = 'fire_r',
    s = 50,
    geo = True,
    tiles="OSM",
).opts(title = "years inactive",height=550)

((plot2 * plot1).opts(height=550, title = 'Provider') + plot_4)

Most of TAD stations seem to have stopped recording in January 2024

India (INCOIS) station seem to be exclusive on WebCritech server

### Get Station detailed metadata (NOT FINISHED)

we'd need to get the First Date information, which can be retrieve by scrapping the website: at the URL `https://webcritech.jrc.ec.europa.eu/SeaLevelsDb/Device/1014` (the device number is the `Provider` column) 
which is contained in `html.hqs-js > body > div > div.container > section.row > div#dev-last-details.col-md-5 > dl.dl-horizontal > dd` of the source HTML. 

In [ ]:
from bs4 import BeautifulSoup
# A function to scrape the "First Date" from the given URL
def get_first_date(device_number):
    print('.', end='')
    url = f"https://webcritech.jrc.ec.europa.eu/SeaLevelsDb/Device/{device_number}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find all dl-horizontal elements, which contain dt and dd pairs
        dl_elements = soup.find_all('dl', class_='dl-horizontal')
        for dl in dl_elements:
            # Find all dt elements within the current dl element
            dt_elements = dl.find_all('dt')
            # print(dt_elements)
            for index, dt in enumerate(dt_elements):
                # Check if the dt element's text is 'First date'
                if dt.get_text().strip() == 'First date':
                    # Get the corresponding dd element that follows the dt element
                    first_date_dd = dl.find_all('dd')[index]
                    # print(first_date_dd.get_text().strip())
                    # 1/0
                    return first_date_dd.get_text().strip()
    return None

# Apply the function to each row and store the results in a new column
SL_df['FirstDate'] = SL_df['Id'].apply(get_first_date)

In [ ]:
from searvey.multi import multithread

# rate_limit 
def fetch_SeaLevelDb_station(tmin: pd.Timestamp, tmax: pd.Timestamp, id: int, NSTEPS = 10000000):
    url = f"https://webcritech.jrc.ec.europa.eu/SeaLevelsDb/api/Device/{id}/Data?tMin={tmin.strftime('%Y-%m-%d')}&tMax={tmax.strftime('%Y-%m-%d')}&nPts={NSTEPS}&field=level&mode=CSV"
    stations_response = requests.get(url)
    if stations_response.ok:
        data = json.loads(stations_response.text)  # Use `loads` instead of `load`
        df = pd.DataFrame(data)
        return df
    else:
        print(f"Error: {stations_response.status_code}")
        return pd.DataFrame()


def fetch_TAD_station(tmin: pd.Timestamp, tmax: pd.Timestamp, id: int, NSTEPS = 10000000): 
    url = f"https://webcritech.jrc.ec.europa.eu/TAD_server/api/Data/Get/{id}?tMin={tmin.strftime('%Y-%m-%d')}&tMax={tmax.strftime('%Y-%m-%d')}&nRec={NSTEPS}&mode=CSV"
    stations_response = requests.get(url)
    if stations_response.ok:
        data = io.StringIO(stations_response.text)  # Create a text stream object
        df = pd.read_csv(data)  # Read the DataFrame from the text stream
        return df
    else:
        print(f"Error: {stations_response.status_code}")
        return pd.DataFrame()


def get_TAD_stations(df:pd.DataFrame, device_var: str):
    for s, station in df.iterrows():
        print(s, station)
        1/0


def get_SLDB_stations(df:pd.DataFrame, device_var: str):
    for s, station in df.iloc[::-1].iterrows():
        print(station)
        func_kwargs = []
        for tstep in pd.date_range(pd.Timestamp(2020, 1, 1), station.LastDate, freq='MS'):
            tmin = tstep
            tmax = tstep + relativedelta(months=+1)
            func_kwargs.append(
                dict(
                    tmin=tmin,
                    tmax=tmax,
                    Device=station[device_var],
                ),
            )
        results = multithread(
            func=fetch_SeaLevelDb_station,
            func_kwargs=func_kwargs,
            n_workers=10,
            print_exceptions=False,
            disable_progress_bar=False,
        )   
        for result in results:
            if result.result is not None:
                df = result.result
                print(df)
        1/0

get_SLDB_stations(SL_df, "Id")

In [27]:
Device = 94
url = f"https://webcritech.jrc.ec.europa.eu/TAD_server/api/Data/Get/{Device}?tMin=2024-08-01&tMax=2024-09-01&nRec={10000000}&mode=CSV"

In [ ]:
stations_response = requests.get(url)
data = io.StringIO(stations_response.text)  # Create a text stream object
df = pd.read_csv(data)  # Read the DataFrame from the text stream
df

In [ ]:
df.hvplot.line( y='Lev RAD (m)')